In [11]:
# antenna specific frequency
import typing
from pydantic import BaseModel, Field


Delta: typing.TypeAlias = int
VectorDelta: typing.TypeAlias = tuple[int, int]
AntiNode: typing.TypeAlias = tuple[int, int]


class Point(BaseModel):
    value: str = Field(min_length=1, max_length=1, description="Value of point in a map")
    x: int
    y: int

    def __eq__(self, point: "Point") -> bool:
        return self.x == point.x and self.y == point.y

    model_config = {
        "frozen": True  # to be able to be added to set
    }


UniqueAntiNodes: typing.TypeAlias = set[Point]


class Map(BaseModel):
    data: list[Point]
    min_point: Point = Point(value=".", x=0, y=0)
    max_point: Point | None = None

    def __init__(self, *args, **kwargs):
        if "data" not in kwargs:
            kwargs["data"] = []
        super().__init__(*args, **kwargs)

    def has_point(self, point: Point) -> bool | Point:
        for _point in self.data:
            if _point == point:
                break
        else:
            return False
        return _point

    def get_point(self, x: int, y: int) -> Point:
        point = Point(value=".", x=x, y=y)
        if _point := self.has_point(point):
            return _point
        raise ValueError("Map has no such a point: ", point)

    def is_antenna(self, point: Point) -> bool | str:
        if _point := self.has_point(point):
            if _point.value != ".":
                return _point.value
        return False

    def is_out_of_map(self, point: Point) -> bool:
        if point.x < self.min_point.x or point.y < self.min_point.y:
            return True
        if self.max_point is not None:
            if point.x > self.max_point.x or point.y > self.max_point.y:
                return True
        return False

    def add_point(self, point: Point):
        if self.has_point(point):
            raise ValueError("Map has already this point: ", point)
        if self.is_out_of_map(point):
            raise ValueError("Trying add point out of map: ", point)
        self.data.append(point)

    def map_completed(self):
        x: int = 0
        y: int = 0
        for point in self.data:
            if x < point.x:
                x = point.x
            if y < point.y:
                y = point.y
        else:
            self.min_point = self.get_point(0, 0)
            self.max_point = self.get_point(x, y)


class Vector(BaseModel):
    start: Point
    end: Point
    delta: VectorDelta | None = None

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.delta = self.count_delta()

    def count_delta(self) -> VectorDelta:
        delta_x: Delta = self.end.x - self.start.x
        delta_y: Delta = self.end.y - self.start.y
        return (delta_x, delta_y)

    def count_antinode(self, mapp: Map) -> list[AntiNode]:
        antinodes: list[AntiNode] = []
        top_antinode: AntiNode = self.start.x - self.delta[0], self.start.y - self.delta[1]
        bottom_antinode: AntiNode = self.end.x + self.delta[0], self.end.y + self.delta[1]
        for node in [top_antinode, bottom_antinode]:
            try:
                point: Point = mapp.get_point(node[0], node[1])
                antinodes.append(point)
            except ValueError:
                # no top antinode here
                pass
        return antinodes


FilePath: typing.TypeAlias = str
StreamOfLines: typing.TypeAlias = typing.Iterator[str]


class OpenStreamOfLines(typing.Protocol):
    def __call__(self, file: FilePath) -> StreamOfLines: ...


def open_stream_of_lines(file: FilePath) -> StreamOfLines:
    with open(file) as file_handler:
        yield from file_handler

In [12]:
from rich import print
mapp = Map()
mapp.add_point(Point(value=".", x=0, y=0))
mapp.add_point(Point(value="A", x=1, y=0))
try:
    mapp.add_point(Point(value=".", x=-1, y=0))
except ValueError as exc:
    print(exc)
try:
    mapp.add_point(Point(value=".", x=0, y=0))
except ValueError as exc:
    print(exc)
print(mapp.has_point(Point(value=".", x=1, y=1)))
print(mapp.has_point(Point(value="A", x=0, y=0)))
print(mapp.get_point(0,0))
print(mapp.get_point(1,0))
try:
    mapp.get_point(1,1)
except ValueError as exc:
    print(exc)
print(mapp.data)
print(mapp.is_out_of_map(Point(value=".", x=-1, y=0)))
print(mapp.is_out_of_map(Point(value=".", x=10, y=10)))
mapp.map_completed()
print(mapp.is_out_of_map(Point(value=".", x=10, y=10)))
print(mapp.is_antenna(Point(value=".", x=1, y=0)))
vector = Vector(start=Point(value=".", x=10, y=5), end=Point(value=".", x=5, y=6))
print(vector)
print(vector.count_antinode(mapp))

('Trying add point out of map: ', Point(value='.', x=-1, y=0))

('Map has already this point: ', Point(value='.', x=0, y=0))

False

Point(value='.', x=0, y=0)

Point(value='.', x=0, y=0)

...

False

True

A

Vector(start=Point(value='.', x=10, y=5), end=Point(value='.', x=5, y=6), delta=(-5, 1))

[]

In [13]:
mapp = Map()
for row, line in enumerate(open_stream_of_lines("../media/2024-day-8.input")):
    for col, _char in enumerate(line):
        if _char != "\n":
            mapp.add_point(Point(value=_char, x=col, y=row))
mapp.map_completed()

vectors: list[Vector] = []
for start_idx, start_point in enumerate(mapp.data):
    for end_point in mapp.data[start_idx + 1:]:
        if start_point.value == end_point.value and start_point.value != ".":
            vectors.append(Vector(start=start_point, end=end_point))

antinodes: UniqueAntiNodes = set()
for vector in vectors:
    for antinode in vector.count_antinode(mapp):
        antinodes.add(antinode)

# print(vectors)
print("antinodes: ", len(antinodes))

antinodes:  259